In [8]:
from newsSentimentScoring import newsLineToScore
import pandas as pd

In [4]:
test_headlines = [
    "Company X reports record-breaking quarterly earnings, exceeding analyst expectations.",
    "Tech giant Y announces a major partnership with Z, driving stock prices up by 15%.",
    "Startup A secures $50 million in funding to expand its innovative product line.",
    "Retailer B sees a surge in sales during the holiday season, forecasting strong growth.",
    "Bank C receives regulatory approval for its new digital banking platform, boosting investor confidence.",
    "Company D's stock plummets after reporting significant losses in the third quarter.",
    "Major layoffs announced at Corporation E as it struggles to cope with declining revenues.",
    "Retailer F faces backlash over supply chain issues, resulting in a 10% drop in sales.",
    "Energy firm G fined $20 million for environmental violations, causing shares to fall.",
    "Tech startup H files for bankruptcy after failing to secure additional funding."
]

In [5]:
cnn_sentiments = newsLineToScore(test_headlines,"CNN")

100%|██████████| 10/10 [00:00<00:00, 2367.39it/s]


['company x reports recordbreaking quarterly earnings exceeding analyst expectations', 'tech giant announces major partnership z driving stock prices 15', 'startup secures 50 million funding expand innovative product line', 'retailer b sees surge sales holiday season forecasting strong growth', 'bank c receives regulatory approval new digital banking platform boosting investor confidence', 'company ds stock plummets reporting significant losses third quarter', 'major layoffs announced corporation e struggles cope declining revenues', 'retailer f faces backlash supply chain issues resulting 10 drop sales', 'energy firm g fined 20 million environmental violations causing shares fall', 'tech startup h files bankruptcy failing secure additional funding']
[[4, 5522, 55, 2936, 660, 24, 10762, 237, 289], [563, 1371, 201, 280, 1528, 7302, 1518, 2, 88, 167], [1853, 2292, 403, 11, 1715, 744, 1769, 414, 221], [872, 769, 171, 372, 5, 672, 1324, 1416, 118, 40], [47, 639, 2103, 2682, 864, 9, 771, 13

In [6]:
lstm_sentiments = newsLineToScore(test_headlines,"LSTM")

100%|██████████| 10/10 [00:00<00:00, 2758.87it/s]


['company x reports recordbreaking quarterly earnings exceeding analyst expectations', 'tech giant announces major partnership z driving stock prices 15', 'startup secures 50 million funding expand innovative product line', 'retailer b sees surge sales holiday season forecasting strong growth', 'bank c receives regulatory approval new digital banking platform boosting investor confidence', 'company ds stock plummets reporting significant losses third quarter', 'major layoffs announced corporation e struggles cope declining revenues', 'retailer f faces backlash supply chain issues resulting 10 drop sales', 'energy firm g fined 20 million environmental violations causing shares fall', 'tech startup h files bankruptcy failing secure additional funding']
[[4, 5522, 55, 2936, 660, 24, 10762, 237, 289], [563, 1371, 201, 280, 1528, 7302, 1518, 2, 88, 167], [1853, 2292, 403, 11, 1715, 744, 1769, 414, 221], [872, 769, 171, 372, 5, 672, 1324, 1416, 118, 40], [47, 639, 2103, 2682, 864, 9, 771, 13

2024-05-28 17:38:50.770316: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-05-28 17:38:50.771852: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-05-28 17:38:50.773390: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 1s 675ms/step


2024-05-28 17:38:51.958858: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-05-28 17:38:51.960378: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-05-28 17:38:51.962062: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [ ]:
results = pd.DataFrame({
    'News':test_headlines,
    'CNN':cnn_sentiments,
    'LSTM':lstm_sentiments,
    "Real":["Positive", "Positive", "Positive", "Positive", "Positive", "Negative", "Negative", "Negative", "Negative", "Negative"]
})